In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from dataloader import *
from model import *
import numpy as np
import os
import pickle
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

config = yaml.safe_load(open('config.yaml', 'r'))
# batch_size = config['params']['batch_size']
batch_size = 1

train_dataset_path = "data/train_dataset"
test_dataset_path = "data/test_dataset"

train_dataset = Dataset(config['path']['data_dir'])
if os.path.exists(train_dataset_path):
    with open(train_dataset_path, "rb") as f:
        train_dataset = pickle.load(f)
else:
    train_dataset.load()
    with open(train_dataset_path, "wb") as f:
        pickle.dump(train_dataset, f)


test_dataset = Dataset(config['path']['test_dir'])
if os.path.exists(test_dataset_path):
    with open(test_dataset_path, "rb") as f:
        test_dataset = pickle.load(f)
else:
    test_dataset.load()
    with open(test_dataset_path, "wb") as f:
        pickle.dump(test_dataset, f)

beat_list = []
track = 0
for midi in train_dataset.encoded_data_list:
    beat_list += midi[track]
train_data = beat_list.copy()

beat_list = []
for midi in test_dataset.encoded_data_list:
    beat_list += midi[track]
test_data = beat_list.copy()

mySet = set(train_data) | set(test_data)
vocab_size = len(mySet)
print(np.array(beat_list).shape)

beat2idx = {beat: i for i, beat in enumerate(mySet)}
idx2beat = {idx:char for char,idx in beat2idx.items()}

def __split_input_target(chunk):
    global vocab_size
    input_text = chunk[:-1]
    target_text = tf.one_hot(chunk[1:], vocab_size)
    return input_text, target_text


idxOfBeat = np.array([beat2idx[beat] for beat in train_data])
print(len(idxOfBeat))
tfDataset = tf.data.Dataset.from_tensor_slices(idxOfBeat)
sequences = tfDataset.batch(config['params']['sequence_length']+1, drop_remainder=True)
# o o o o o
#  \ \ \ \ \
#   x x x x x
ds = sequences.map(__split_input_target)
train_data = ds.shuffle(10000).batch(batch_size, drop_remainder=True)


idxOfBeat = np.array([beat2idx[beat] for beat in test_data])
print(len(idxOfBeat))
tfDataset = tf.data.Dataset.from_tensor_slices(idxOfBeat)
sequences = tfDataset.batch(config['params']['sequence_length']+1, drop_remainder=True)
# o o o o o
#  \ \ \ \ \
#   x x x x x
ds = sequences.map(__split_input_target)
test_data = ds.shuffle(10000).batch(batch_size, drop_remainder=True)

print(f'\nTraining dataset : {train_data}\n')


model = LoFiLoopNet('dev', vocab_size, batch_size=1)
model.summary()

(7429,)
23674
7429

Training dataset : <BatchDataset shapes: ((1, 256), (1, 256, 296)), types: (tf.int64, tf.float32)>

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            75776     
_________________________________________________________________
lstm (LSTM)                  (1, None, 512)            1574912   
_________________________________________________________________
batch_normalization (BatchNo (1, None, 512)            2048      
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 512)            2099200   
_________________________________________________________________
batch_normalization_1 (Batch (1, None, 512)            2048      
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 512)            2099200   
__

In [2]:
model.load_weights('models/model-stateful-1855-1.5676-bigger.hdf5')
model.model.build(tf.TensorShape([1, None]))

Loaded weight models/model-stateful-1855-1.5676-bigger.hdf5


In [3]:
loaded_model = model.model

In [4]:
td = test_data.enumerate()
init_x = None
init_y = None
for index, (x, y) in td.as_numpy_iterator():
    init_x = tf.expand_dims([x[0, 0]], 0)
    init_y = tf.expand_dims([y[0, 0]], 0)
    break


In [5]:
init_x

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[77]], dtype=int32)>

In [6]:
loaded_model.reset_states()
pred = loaded_model(init_x)

In [7]:
music_sequence = []

for i in range(100):
    prediction = tf.squeeze(pred, 0).numpy()
    top_k = tf.math.top_k(prediction, 3)
    top_k_choices = top_k[1].numpy().squeeze()
    top_k_values = top_k[0].numpy().squeeze()

    # Apply random
    next_beat = None
    if np.random.uniform(0, 1) < .5:
        next_beat = top_k_choices[0]
    else:
        p_choices = tf.math.softmax(top_k_values[1:]).numpy()
        next_beat = np.random.choice(top_k_choices[1:], 1, p=p_choices)[0]

    music_sequence.append(idx2beat[next_beat])

    expanded_input = tf.expand_dims([next_beat], 0)
    loaded_model(expanded_input)


In [ ]:
new_midi = MidiData()
new_midi.decode(music_sequence, filename='stateful-2-l1.5.mid')